# Downsample the point clouds
source_down = source.voxel_down_sample(voxel_size=voxel_size)
target_down = target.voxel_down_sample(voxel_size=voxel_size)

# Estimate normals of downsampled point clouds
source_down.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(
    radius=voxel_size*2, max_nn=30))
target_down.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(
    radius=voxel_size*2, max_nn=30))

# Compute FPFH features for downsampled point clouds
source_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
    source_down, o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size*5, max_nn=100))
target_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
    target_down, o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size*5, max_nn=100))
# Global registration with RANSAC
We are going to use [open3d](http://www.open3d.org/) to handle point clouds and generation of point clouds
We are importing the packages and defining a function which helps us drawing the point clouds.

In [20]:
import open3d as o3d
import numpy as np
import copy

# helper function for drawing
# If you want it to be more clear set recolor=True
def draw_registrations(source, target, transformation = None, recolor = False):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    if(recolor):
        source_temp.paint_uniform_color([1, 0.706, 0])
        target_temp.paint_uniform_color([0, 0.651, 0.929])
    if(transformation is not None):
        source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])

We need to read in our pointclouds. For that we use the `io` module of the
open3d package (`o3d`). The following cell will open a window with a
visualization of both point clouds `source` and `target`.

Also, this page [Visualization - Open3D](http://open3d.org/html/tutorial/Basic/visualization.html)
contains some useful examples and instructions on how to use the viewer.

In [26]:
source = o3d.io.read_point_cloud("ICP/r1.pcd")
target = o3d.io.read_point_cloud("ICP/r2.pcd")

# Used for downsampling.
voxel_size = 0.05

# Show models side by side
draw_registrations(source, target)

### Finding features in pointclouds
When working on point clouds it can be beneficial to work on a downsampled version of the point cloud,
as it decreases the need of computation.

You can use [`pointcloud.voxel_down_sample()`](http://www.open3d.org/docs/latest/python_api/open3d.geometry.PointCloud.html#open3d.geometry.PointCloud.voxel_down_sample) where `pointcloud` is the name of your point cloud object. In our case, that would be `source` and `target`.

We also need to estimate the normals of the point cloud points using [`pointcloud.estimate_normals()`](http://www.open3d.org/docs/latest/python_api/open3d.geometry.PointCloud.html#open3d.geometry.PointCloud.voxel_down_sample)

**Task:** Find FPFH features or correspondances of the downsampled point clouds.
[`o3d.pipelines.registration.compute_fpfh_feature()`](http://www.open3d.org/docs/latest/python_api/open3d.pipelines.registration.compute_fpfh_feature.html)


In [31]:
####
# Downsample and find features here
####

# # Code
source_down = source.voxel_down_sample(voxel_size=0.05)
target_down = target.voxel_down_sample(voxel_size=0.05)
# draw_registrations(source_down, target_down)
#
source_down.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
target_down.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))

#
source_fpfh = o3d.pipelines.registration.compute_fpfh_feature(source_down, o3d.geometry.KDTreeSearchParamHybrid(radius=0.25, max_nn=100))
target_fpfh = o3d.pipelines.registration.compute_fpfh_feature(target_down, o3d.geometry.KDTreeSearchParamHybrid(radius=0.25, max_nn=100))
#
#correspondences = o3d.pipelines.registration.correspondence_feature_based(source_fpfh, target_fpfh, mutual_filter=True)
draw_registrations(source_down, target_down)



### RANSAC 
We will now attempt to use RANSAC to do a global registration of the two point clouds.

By using the function [`o3d.pipelines.registration.registration_ransac_based_on_feature_matching`](http://www.open3d.org/docs/latest/python_api/open3d.pipelines.registration.registration_ransac_based_on_feature_matching.html) from open3d, do the following:


Try to find the transformation from `r1.pcd` (`source`) to `r2.pcd` (`target`).
Attempt with point-to-point and point-to-plane
```Python
point_to_point =  o3d.pipelines.registration.TransformationEstimationPointToPoint(False)
point_to_plane =  o3d.pipelines.registration.TransformationEstimationPointToPlane()
```

When using RANSAC, focus on the arguments below. The rest are optional parameters.
```Python
ransac_result = o3d.registration.registration_ransac_based_on_feature_matching(
    source_sample, target_sample, 
    source_fpfh, target_fpfh, 
    distance_threshold,
    point_to_point)
```

In [32]:
# Set the RANSAC parameters

point_to_point = o3d.pipelines.registration.TransformationEstimationPointToPoint(False)
point_to_plane = o3d.pipelines.registration.TransformationEstimationPointToPlane()

# Set the distance threshold for RANSAC
distance_threshold = 0.075

# Perform RANSAC-based registration using point-to-point method
ransac_result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
    source_down, target_down, 
    source_fpfh, target_fpfh, 
    True,
    distance_threshold,
    point_to_point)

# Get the transformation matrix
transformation_matrix = ransac_result.transformation

# Print the transformation matrix
print(transformation_matrix)

# Apply the transformation to the source point cloud
source_transformed = source_down.transform(transformation_matrix)

# Visualize the registration result
draw_registrations(source_transformed, target_down)


[[-0.53489662  0.84475731  0.0164525   0.58954274]
 [-0.2374326  -0.16897194  0.95659513  0.91449413]
 [ 0.81087075  0.50777314  0.29095544 -1.50147516]
 [ 0.          0.          0.          1.        ]]


## Exercises
### A)
Can you get a decent transformation from r1 to r3? (check the ICP folder)
### B)
With the following checkers, can you get better results from RANSAC? Try tweaking the parameters of them. Can you make point-to-plane work? Do not spend too much time on this, if you can't manage, skip it. (I was not able to get a good fit.)


You can also try tweaking the `voxel_size`

```Python
corr_length = 0.9
distance_threshold = voxel_size * 1.5

c0 = o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(corr_length)
c1 = o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold)
c2 = o3d.pipelines.registration.CorrespondenceCheckerBasedOnNormal(0.095)

checker_list = [c0,c1,c2]

ransac_result = o3d.registration.registration_ransac_based_on_feature_matching(
    source_sample, target_sample, 
    source_fpfh, target_fpfh, 
    True,
    distance_threshold,
    point_to_point,
    checkers = checker_list)
```


In [14]:
# A
# Read in the point clouds
source = o3d.io.read_point_cloud("ICP/r1.pcd")
target = o3d.io.read_point_cloud("ICP/r3.pcd")

# Downsample the point clouds
voxel_size = 0.05
source_down = source.voxel_down_sample(voxel_size)
target_down = target.voxel_down_sample(voxel_size)

# Estimate the normals of the downsampled point clouds
source_down.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
target_down.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))

# Compute FPFH features
source_fpfh = o3d.pipelines.registration.compute_fpfh_feature(source_down,
    o3d.geometry.KDTreeSearchParamHybrid(radius=0.25, max_nn=100))
target_fpfh = o3d.pipelines.registration.compute_fpfh_feature(target_down,
    o3d.geometry.KDTreeSearchParamHybrid(radius=0.25, max_nn=100))

# Set the RANSAC parameters
distance_threshold = voxel_size * 1.5
point_to_point = o3d.pipelines.registration.TransformationEstimationPointToPoint(False)
point_to_plane = o3d.pipelines.registration.TransformationEstimationPointToPlane()

# Perform RANSAC-based global registration
ransac_result_p2p = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
    source_down, target_down, source_fpfh, target_fpfh,True,
    distance_threshold, point_to_point)

# Print the transformation matrix
print(ransac_result_p2p.transformation)

# Apply the transformation to the source point cloud
source_transformed = source_down.transform(ransac_result_p2p.transformation)

# Visualize the registration result
draw_registrations(source_transformed, target_down)

[[-0.23227502  0.95713942 -0.17300995  0.33002872]
 [ 0.26850882  0.23406178  0.93440789 -0.41665994]
 [ 0.93485364  0.17058491 -0.31136708  0.07577581]
 [ 0.          0.          0.          1.        ]]


In [17]:
#B
import open3d as o3d
import numpy as np
import copy
#
corr_length = 0.9
distance_threshold = voxel_size * 1.5

c0 = o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(corr_length)
c1 = o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold)
c2 = o3d.pipelines.registration.CorrespondenceCheckerBasedOnNormal(0.095)

checker_list = [c0,c1,c2]

ransac_result_b = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
    source_down, target_down, 
    source_fpfh, target_fpfh, 
    True,
    distance_threshold,
    point_to_point,
    checkers = checker_list)
checker_list
transformation_matrix = ransac_result_b.transformation
print(transformation_matrix)

# Apply the transformation to the source point cloud
source_transformed = source_down.transform(transformation_matrix)

# Visualize the registration result
draw_registrations(source_transformed, target_down)

[[ 0.99845899 -0.05487709  0.00825535  0.07861426]
 [ 0.05541871  0.99377485 -0.09664533  0.02448248]
 [-0.00290035  0.0969539   0.99528465 -0.17741354]
 [ 0.          0.          0.          1.        ]]
